# 🎮 GPU-as-a-Service Orchestrator

**Platform Engineering Standard Operating Procedure (SOP)**

This notebook demonstrates **RHOAI 3.0 GPU quota management** using native OpenShift `oc` commands that any Platform Engineer can use.

---

## 📊 Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    Kueue Quota Management                       │
│                                                                 │
│   ClusterQueue: rhoai-main-queue                                │
│   ├── nominalQuota: 5 GPUs                                      │
│   └── ResourceFlavors: nvidia-l4-1gpu, nvidia-l4-4gpu           │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📺 Setup: Open Monitoring Dashboard

**Before running the demo**, open the RHOAI Dashboard in another tab:

1. Navigate to **RHOAI Dashboard** → **Distributed Workloads**
2. Select project: **private-ai**
3. Watch the **Workloads** table for status changes

This provides real-time visibility into Kueue admission decisions.


---

## 🔍 Step 0: Check Current State

Let's verify the baseline: **5/5 GPUs used** (BF16 + INT4)


In [ ]:
# View all InferenceServices and their replica counts
! oc get inferenceservice -n private-ai


In [ ]:
# Check Kueue workloads - shows admitted vs pending
! oc get workloads -n private-ai


In [ ]:
# Check running pods (should see bf16 and int4)
! oc get pods -n private-ai -l serving.kserve.io/inferenceservice


---

## 🚀 Scenario 1: Resource Conflict

**Story:** "A data scientist requests Devstral-2 for agentic workflows."

We'll attempt to start Devstral-2 (4 GPUs), but the cluster is already at capacity:
- Current: BF16 (4) + INT4 (1) = **5/5 GPUs**
- Requested: Devstral-2 (4) = **+4 GPUs**
- Result: **Over quota** → Kueue holds in PENDING state


In [ ]:
# Scale up Devstral-2 (requests 4 GPUs)
! oc patch inferenceservice devstral-2 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":1}}}'


In [ ]:
# Check workload status - Devstral-2 should be PENDING
! oc get workloads -n private-ai


### 💡 Observation

In the **Distributed Workloads** dashboard, you should see:
- `devstral-2` workload with status: **Pending**
- Message: "Insufficient quota" or "Waiting for resources"

**This is Kueue working correctly!** The request is queued, not rejected.


---

## 🔄 Scenario 2: Dynamic Handover

**Story:** "Platform team hibernates the BF16 model to free GPUs."

We'll scale down BF16 to release 4 GPUs. Kueue will **automatically** admit Devstral-2.

**Watch the dashboard!** This is the "wow moment" of the demo.


In [ ]:
# Scale down BF16 to release 4 GPUs
! oc patch inferenceservice mistral-3-bf16 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":0}}}'


In [ ]:
# Check the transition - BF16 terminates, Devstral-2 starts
! oc get pods -n private-ai -l serving.kserve.io/inferenceservice


In [ ]:
# Verify final state
! oc get inferenceservice -n private-ai
! echo ""
! oc get workloads -n private-ai


### 💡 Key Observation

**Devstral-2 started INSTANTLY** when BF16 was hibernated.

This is **GPU-as-a-Service**: Resources flow to where they're needed, governed by quota.

Current State:
- Devstral-2 (4) + INT4 (1) = **5/5 GPUs**


---

## 📉 Scenario 3: Efficiency Portfolio

**Story:** "Trade one 4-GPU model for multiple specialized 1-GPU models."

Instead of one big model, we'll run:
- `granite-8b-agent` (1 GPU) - Agentic tool-calling
- `gpt-oss-20b` (1 GPU) - High-reasoning *(if weights available)*
- `mistral-3-int4` (1 GPU) - Cost-efficient inference

**Same 5 GPU quota, 3x more model diversity!**


In [ ]:
# Step 1: Hibernate Devstral-2 to free 4 GPUs
! oc patch inferenceservice devstral-2 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":0}}}'


In [ ]:
# Step 2: Start specialized 1-GPU models
! oc patch inferenceservice granite-8b-agent -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":1}}}'


In [ ]:
# Check the new portfolio
! oc get inferenceservice -n private-ai
! echo ""
! oc get workloads -n private-ai


### 💡 The Efficiency Message

**Before:** 1 big model (Devstral-2, 4 GPUs)
**After:** 2 specialized models (INT4 + Granite, 2 GPUs)

You've traded raw power for **specialization and efficiency**, with room to spare!


---

## 🔄 Reset: Return to Baseline State

Return to the original "saturated" state for the next demo run.


In [ ]:
# Reset all models to baseline
! oc patch inferenceservice mistral-3-bf16 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":1}}}'
! oc patch inferenceservice mistral-3-int4 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":1}}}'
! oc patch inferenceservice devstral-2 -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":0}}}'
! oc patch inferenceservice gpt-oss-20b -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":0}}}'
! oc patch inferenceservice granite-8b-agent -n private-ai --type=merge -p '{"spec":{"predictor":{"minReplicas":0}}}'


In [ ]:
# Verify baseline restored
! oc get inferenceservice -n private-ai


---

## 📖 Technical Reference

### Why `oc patch` (not `oc scale`) for KServe

KServe InferenceService does **not** implement the Kubernetes scale subresource in RHOAI 3.0.
Use `oc patch` to modify `spec.predictor.minReplicas`:

```bash
# Scale up (minReplicas=1 starts the model)
oc patch inferenceservice <name> -n private-ai --type=merge \
  -p '{"spec":{"predictor":{"minReplicas":1}}}'

# Scale down (minReplicas=0 hibernates the model)
oc patch inferenceservice <name> -n private-ai --type=merge \
  -p '{"spec":{"predictor":{"minReplicas":0}}}'
```

### How Kueue Intercepts Scale Events

Because the `private-ai` namespace has `kueue.openshift.io/managed=true`:

1. Patch event modifies `minReplicas` → triggers pod creation request
2. Kueue intercepts via admission webhook
3. Checks against `ClusterQueue` quota (5 GPUs)
4. **If under quota:** Workload admitted immediately
5. **If over quota:** Workload held in PENDING until resources free

### Documentation Links

- [RHOAI 3.0: Deploying Models](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/3.0/html-single/deploying_models/index)
- [RHOAI 3.0: Managing Distributed Workloads](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/3.0/html-single/working_with_distributed_workloads/index)
- [Kueue Documentation](https://kueue.sigs.k8s.io/)


---

## 🎤 Demo Script (Presenter Notes)

### Opening (30 seconds)
> "In enterprise environments, GPU resources are expensive and shared. Teams can't hoard GPUs indefinitely. Let me show you how RHOAI 3.0 manages this with **GPU-as-a-Service**."

### Scenario 1: The Conflict (1 minute)
> "We have 5 GPUs and a 5 GPU quota. Right now, Mistral BF16 uses 4, and INT4 uses 1. That's 100% utilization."
> 
> "A data scientist requests Devstral-2 for agentic workflows. Let's see what happens..."
> 
> *Run scale command*
> 
> "Notice it's PENDING—not rejected. Kueue is holding it in queue until resources are available."

### Scenario 2: The Handover (1 minute)
> "As a Platform Engineer, I can hibernate the BF16 model to free those 4 GPUs."
> 
> *Run scale down command*
> 
> "Watch the dashboard—Devstral-2 starts INSTANTLY. This is the magic of Kueue."

### Closing (30 seconds)
> "This is GPU-as-a-Service. Model Registry provides governance, Kueue provides resource arbitration. Organizations access a wide range of Red Hat Validated models while controlling AWS GPU costs."
